# 3) Mutability — Exercises

**Learning goals:** reference semantics, aliasing, in-place vs copy, shallow vs deep copy, default mutable args, safe patterns.

### Warm-ups

1. **In-place or copy?** — Return a **new** list with x appended; don’t mutate input.

```python
def append_copy(xs, x):
    ...
a = [1,2]; b = append_copy(a, 3)
assert a == [1,2] and b == [1,2,3]
```

2. **Safe default parameter**

```python
def append_safe(x, bucket=None):
    """Append x to bucket but avoid shared default list."""
    ...
b1 = append_safe(1); b2 = append_safe(2)
assert b1 == [1] and b2 == [2]
```

3. **Clone 2-D list (shallow vs deep)**

```python
def shallow_clone_2d(m): ...
def deep_clone_2d(m): ...
m = [[1],[2]]
m2 = shallow_clone_2d(m); m3 = deep_clone_2d(m)
m[0].append(9)
assert m2[0] == [1,9] and m3[0] == [1]
```

### Core

4. **Freeze nested** (list/tuple/dict → immutables)

```python
def freeze(obj):
    """Convert lists to tuples, dicts to tuples of (key, frozen(value)) sorted by key."""
    ...
f = freeze({"a":[1,2], "b":{"x":1}})
assert isinstance(f, tuple) and ("a", (1,2)) in f
```

5. **Deep update (write-on-copy)**
   Merge dict `updates` into dict `base` without mutating `base`; nested dicts should merge recursively.

```python
def deep_merge(base, updates):
    ...
base = {"a":1,"b":{"x":1}}
u = {"b":{"y":2},"c":3}
merged = deep_merge(base,u)
assert base == {"a":1,"b":{"x":1}}
assert merged == {"a":1,"b":{"x":1,"y":2},"c":3}
```

6. **Sort safely** — Return a sorted copy by key; original list of dicts stays unchanged.

```python
def sort_users_by(users, key):
    ...
U = [{"name":"b","age":30},{"name":"a","age":20}]
out = sort_users_by(U,"name")
assert out[0]["name"]=="a" and U[0]["name"]=="b"
```

7. **Toggle flag in dict** — don’t mutate input

```python
def toggled(d, key):
    """Return a copy with boolean d[key] toggled (missing -> True)."""
    ...
assert toggled({"a":False},"a") == {"a":True}
```

### Challenge

8. **deep\_copy\_except(keys\_to\_skip)**
   Deep-copy a nested structure but **preserve references** for subtrees whose top-level keys are in `keys_to_skip` (for dicts).

```python
def deep_copy_except(obj, keys_to_skip=frozenset()):
    ...
src = {"cfg":{"x":1}, "cache":[1,2]}
out = deep_copy_except(src, {"cache"})
assert out["cfg"] is not src["cfg"] and out["cache"] is src["cache"]
```


In [9]:
# 1) In-place or copy?  (return new list, don't mutate input)
def append_copy(xs, x):
    return xs + [x]

a = [1,2]; b = append_copy(a, 3)
assert a == [1,2] and b == [1,2,3]

In [10]:
# 2) Safe default parameter
def append_safe(x, bucket=None):
    if bucket is None:
        bucket = []
    bucket.append(x)
    return bucket

b1 = append_safe(1); b2 = append_safe(2)
assert b1 == [1] and b2 == [2]

In [11]:
# 3) Clone 2-D list (shallow vs deep)
def shallow_clone_2d(m):
    return m[:]

def deep_clone_2d(m):
    return [row[:] for row in m]

m = [[1],[2]]
m2 = shallow_clone_2d(m); m3 = deep_clone_2d(m)
m[0].append(9)
assert m2[0] == [1,9] and m3[0] == [1]

In [12]:
# 4) Freeze nested (list/tuple/dict -> immutables)
def freeze(obj):
    if isinstance(obj, dict):
        return tuple((k, freeze(v)) for k, v in sorted(obj.items(), key=lambda kv: kv[0]))
    if isinstance(obj, list):
        return tuple(freeze(x) for x in obj)
    if isinstance(obj, tuple):
        return tuple(freeze(x) for x in obj)
    if isinstance(obj, set):
        return tuple(sorted(freeze(x) for x in obj))
    return obj

f = freeze({"a":[1,2], "b":{"x":1}})
assert isinstance(f, tuple) and ("a", (1,2)) in f

In [13]:
# 5) Deep update (write-on-copy)
def deep_merge(base, updates):
    if not isinstance(base, dict) or not isinstance(updates, dict):
        return updates
    out = {k: v for k, v in base.items()}  # shallow copy
    for k, v in updates.items():
        if k in out and isinstance(out[k], dict) and isinstance(v, dict):
            out[k] = deep_merge(out[k], v)  # recurse
        else:
            out[k] = v
    return out

base = {"a":1,"b":{"x":1}}
u = {"b":{"y":2},"c":3}
merged = deep_merge(base,u)
assert base == {"a":1,"b":{"x":1}}
assert merged == {"a":1,"b":{"x":1,"y":2},"c":3}

In [14]:
# 6) Sort safely — return a sorted copy by key; don't mutate original
def sort_users_by(users, key):
    return sorted(users, key=lambda u: u[key])

U = [{"name":"b","age":30},{"name":"a","age":20}]
out = sort_users_by(U,"name")
assert out[0]["name"]=="a" and U[0]["name"]=="b"

In [15]:
# 7) Toggle flag in dict — don’t mutate input
def toggled(d, key):
    out = dict(d)
    out[key] = not d.get(key, False)
    return out

assert toggled({"a":False},"a") == {"a":True}

In [16]:
# 8) deep_copy_except(keys_to_skip)
def deep_copy_except(obj, keys_to_skip=frozenset()):
    """
    Deep-copy a nested structure but preserve references for subtrees
    whose dict-keys are in keys_to_skip (applies at every dict level).
    """
    # primitives & immutables
    if isinstance(obj, (int, float, str, bytes, bool, type(None))):
        return obj
    # dict: copy unless key is in skip
    if isinstance(obj, dict):
        out = {}
        for k, v in obj.items():
            if k in keys_to_skip:
                out[k] = v                 # preserve reference
            else:
                out[k] = deep_copy_except(v, keys_to_skip)
        return out
    # list/tuple/set/frozenset
    if isinstance(obj, list):
        return [deep_copy_except(x, keys_to_skip) for x in obj]
    if isinstance(obj, tuple):
        return tuple(deep_copy_except(x, keys_to_skip) for x in obj)
    if isinstance(obj, set):
        return {deep_copy_except(x, keys_to_skip) for x in obj}
    if isinstance(obj, frozenset):
        return frozenset(deep_copy_except(x, keys_to_skip) for x in obj)
    # fallback: return as-is (unknown types)
    return obj

src = {"cfg":{"x":1}, "cache":[1,2]}
out = deep_copy_except(src, {"cache"})
assert out["cfg"] is not src["cfg"] and out["cache"] is src["cache"]